In [1]:
import pandas as pd

# Carregando os dados de treinamento e teste
train_df = pd.read_csv("train_critic.csv").dropna()  # Carregando dados de treino
train_df = train_df.groupby('Sentiment', group_keys=False).apply(lambda x: x.sample(1500)).reset_index(drop=True)

/tmp/ipykernel_30184/2074655177.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train_df = train_df.groupby('Sentiment', group_keys=False).apply(lambda x: x.sample(1500)).reset_index(drop=True)


In [2]:
from transformers import AutoTokenizer, Trainer, Trainer, TrainingArguments, AutoModelForSequenceClassification
import torch

NUM_LABELS = 2  # Número de classes
model_name = "google-bert/bert-base-uncased"  # Escolha o modelo conforme sua necessidade
tokenizer = AutoTokenizer.from_pretrained(model_name)  # Carregar o tokenizer
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=NUM_LABELS)  # Carregar o modelo de classificação

/home/felipedias/BigData/Trabalho1/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-23 12:15:39.148376: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-23 12:15:39.155510: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732374939.164355   30184 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732374939.167101   30184 cuda_bla

In [3]:
from datasets import Dataset
import tensorflow as tf
import random
import numpy as np
tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)


train_data_set = Dataset.from_pandas(train_df)
train_val_set = train_data_set.train_test_split (test_size = 0.2)

print(train_val_set)

x_train = train_val_set["train"]["Review"]
x_val = train_val_set["test"]["Review"]
y_train = train_val_set["train"]["Sentiment"]
y_val = train_val_set["test"]["Sentiment"] 



DatasetDict({
    train: Dataset({
        features: ['Show', 'Sentiment', 'Review'],
        num_rows: 2400
    })
    test: Dataset({
        features: ['Show', 'Sentiment', 'Review'],
        num_rows: 600
    })
})


In [ ]:
base_train_encodings = tokenizer(list(x_train), truncation=True, padding=True, max_length=60)
base_valid_encodings = tokenizer(list(x_val), truncation=True, padding=True, max_length=60)

In [5]:
from sklearn.metrics import balanced_accuracy_score

def compute_metrics(p):
    return {"balanced_accuracy": balanced_accuracy_score(p.label_ids, np.argmax(p.predictions, axis=1))}

In [6]:
import seaborn as sns
import matplotlib.pyplot as plt

def  plot_loss_accuracy(df):
    plt.figure(figsize=(16, 6))
    plt.subplot(121)
    sns.lineplot(data=df[['loss', 'eval_loss']])

In [7]:
from sklearn import metrics

def plot_confusion_matrix(dataset, predictions, labels=None):
    #calcula a acurácia balanceada
    labels = dataset['Sentiment']
    preds = [0 if pred['label'] == 'LABEL_0' else 1 for pred in predictions]
    
    
    print('balanced Accuracy:', balanced_accuracy_score(labels, preds))
    print('accuracy', metrics.accuracy_score(labels, preds))

    # Calculo da matriz de confusão.
    c = metrics.confusion_matrix(labels, preds)
    r = c / c.sum(axis=1, keepdims=True)

    plt.figure(figsize=(10, 8))
    sns.heatmap(r, annot=True, fmt=".2f", cmap="Blues")
    plt.xlabel("Classe predita")
    plt.ylabel("Classe verdadeira")
    plt.title("Matriz de confusão relativa")
    plt.show()

## Modelo Base

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rndForest = RandomForestClassifier(max_depth=8, n_estimators=2, max_features=66)
rndForest.fit(base_train_encodings["input_ids"], y_train)
p_val = rndForest.predict(valid_encodings["input_ids"])

## Modelo Melhorado

In [ ]:
train_encodings = tokenizer(list(x_train), truncation=True, padding=True, max_length=256)
valid_encodings = tokenizer(list(x_val), truncation=True, padding=True, max_length=256)

In [9]:
class DatasetLoader(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)
    
train_dataset = DatasetLoader(train_encodings, list(y_train))
valid_dataset = DatasetLoader(valid_encodings, list(y_val))


In [10]:
training_args = TrainingArguments(
per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=35,
    logging_dir='./logs1',
    logging_steps=200,
    evaluation_strategy="steps",
    eval_steps=200,
    output_dir='./results1',
    learning_rate=0.00001,
    lr_scheduler_type="constant",
    remove_unused_columns=False,
)


/home/felipedias/BigData/Trabalho1/.venv/lib/python3.12/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:
def make_model_contiguous(model):
    # Aplica .contiguous() a todos os tensores dos parâmetros do modelo
    for param in model.parameters():
        if not param.is_contiguous():
            param.data = param.data.contiguous()

In [12]:


make_model_contiguous(model)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

  4%|▍         | 200/5250 [00:19<07:57, 10.58it/s]

{'loss': 0.4655, 'grad_norm': 12.037578582763672, 'learning_rate': 1e-05, 'epoch': 1.33}


                                                  
  4%|▍         | 201/5250 [00:20<18:01,  4.67it/s]

{'eval_loss': 0.3612649738788605, 'eval_balanced_accuracy': 0.8558315033830857, 'eval_runtime': 0.7926, 'eval_samples_per_second': 756.975, 'eval_steps_per_second': 23.971, 'epoch': 1.33}


  8%|▊         | 400/5250 [00:40<08:08,  9.93it/s]

...

 15%|█▌        | 800/5250 [01:22<07:19, 10.13it/s]

{'loss': 0.0573, 'grad_norm': 0.053174879401922226, 'learning_rate': 1e-05, 'epoch': 5.33}


                                                  
 15%|█▌        | 802/5250 [01:23<16:41,  4.44it/s]

{'eval_loss': 0.7133636474609375, 'eval_balanced_accuracy': 0.8613826621038668, 'eval_runtime': 0.8356, 'eval_samples_per_second': 718.051, 'eval_steps_per_second': 22.738, 'epoch': 5.33}


 16%|█▌        | 824/5250 [01:25<07:31,  9.80it/s]

In [ ]:
df_loss = pd.DataFrame(columns=['epoch', 'loss'])
df_val_loss = pd.DataFrame(columns=['epoch', 'eval_loss','eval_balanced_accuracy'])

for values in trainer.state.log_history:
    if values.get('loss') is not None:
         df_new_row = pd.DataFrame.from_records({'epoch': values['epoch'], 'loss': values['loss']}, index=[0])
         df_loss = pd.concat([df_loss, df_new_row], ignore_index=True)
    elif values.get('train_loss') is not None:
         df_new_row = pd.DataFrame.from_records({'epoch': values['epoch'], 'loss': values['train_loss']}, index=[0])
         df_loss = pd.concat([df_loss, df_new_row], ignore_index=True)
    elif values.get('eval_loss') is not None:
         df_val_new_row = pd.DataFrame.from_records({'epoch': values['epoch'], 'eval_loss': values['eval_loss'], 'eval_balanced_accuracy': values['eval_balanced_accuracy']}, index=[0])
         df_val_loss = pd.concat([df_val_loss, df_val_new_row], ignore_index=True)

mergedDf = pd.merge(df_loss, df_val_loss)


In [ ]:
plot_loss_accuracy(mergedDf)

In [ ]:
from transformers import pipeline
classifier = pipeline('text-classification', model=model, tokenizer=tokenizer)

predictions = classifier(train_val_set["test"]["Review"], batch_size=16)


plot_confusion_matrix(train_val_set["test"], predictions)
